In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [20]:
from gensim import corpora
import pickle
from pathlib import Path
from io import FileIO
import pyLDAvis.gensim
from gensim.models import CoherenceModel, LdaModel, LdaMulticore
import pandas as pd
from youbemom import create_connection
import csv
import random
import numpy as np
import math

In [6]:
p = Path.cwd()
path_parent = p.parents[0]

In [8]:
path_lemma_pkl = str(path_parent / "clean_data" / "lemmatized_text_{}.pkl")
path_corpus_pkl = str(path_parent / "clean_data" / "corpus_{}.pkl")
path_dictionary_gensim = str(path_parent / "clean_data" / "dictionary_{}.gensim")
path_model = path_parent / "clean_data"
path_sample = path_parent / "clean_data" / "sample_lda"
# path_db = path_parent / "database" / "youbemomTables.db"
# path_db = str(path_db)
path_coherence = str(path_parent / "clean_data" / "coherence_{}.csv")

In [10]:
lemmatized_text = pickle.load(open(path_lemma_pkl.format('special-needs'), 'rb'))
corpus = pickle.load(open(path_corpus_pkl.format('special-needs'), 'rb'))
dictionary = corpora.Dictionary.load(path_dictionary_gensim.format('special-needs'))

In [21]:
# multicore workers
w = 2

In [22]:
# temperature
initial_temp = 90
final_temp = 88
alpha = 0.01
current_temp = initial_temp

In [23]:
# bounds
# 1. number of topics (int)
# 2. number of iterations (int)
# 3. alpha (float)
# 4. beta (float)
lower_bounds = [2, 5, 0.00001, 0.00001]
upper_bounds = [20, 200, 1.0, 1.0]

In [24]:
# initial state
def initialize_state(lower_bounds, upper_bounds):
#     state = [random.randint(lower_bounds[0], upper_bounds[0]),
#             random.randint(lower_bounds[1], upper_bounds[1])]
#     noise = [random.random() for i in range(2)]
#     state += [l + r * (u - l) for l, u, r in zip(lower_bounds[2:], upper_bounds[2:], noise)]
    noise = [random.random() for i in lower_bounds]
    state = [l + r * (u - l) for l, u, r in zip(lower_bounds, upper_bounds, noise)]
    return state

In [25]:
initial_state = initialize_state(lower_bounds, upper_bounds)
current_state = initial_state.copy()
solution = current_state.copy()

In [26]:
# get neighbors
def get_neighbor(individual, eta, indpb, low, up):
    """Polynomial mutation as implemented in original NSGA-II algorithm in
    C by Deb.
    :param individual: a list of values to be mutated.
    :param eta: Crowding degree of the mutation. A high eta will produce
                a mutant resembling its parent, while a small eta will
                produce a solution much more different.
    :param indpb: Probability of the mutation for each gene. A high p
                  will make genes more likely to mutate and a low p
                  will make genes less likely to mutate.
    :param low: A list of values that is the lower bound of the search
                space.
    :param up: A list of values values that is the upper bound of the
               search space.
    :returns: A list of the mutated state within the given bounds.
    
    adapted from:
    https://github.com/DEAP/deap/blob/master/deap/tools/mutation.py
    """
    size = len(individual)

    for i, xl, xu in zip(range(size), low, up):
        if random.random() <= indpb:
            x = individual[i]
            delta_1 = (x - xl) / (xu - xl)
            delta_2 = (xu - x) / (xu - xl)
            rand = random.random()
            mut_pow = 1.0 / (eta + 1.)

            if rand < 0.5:
                xy = 1.0 - delta_1
                val = 2.0 * rand + (1.0 - 2.0 * rand) * xy ** (eta + 1)
                delta_q = val ** mut_pow - 1.0
            else:
                xy = 1.0 - delta_2
                val = 2.0 * (1.0 - rand) + 2.0 * (rand - 0.5) * xy ** (eta + 1)
                delta_q = 1.0 - val ** mut_pow

            x = x + delta_q * (xu - xl)
            x = min(max(x, xl), xu)
            individual[i] = x
    return individual

In [30]:
# get cost
def get_cost(state):
    n_topics = int(round(state[0]))
    n_iterations = int(round(state[1]))
    a = state[2]
    b = state[3]
    # do LDA
    %time ldamodel = LdaMulticore(corpus, num_topics = n_topics, id2word=dictionary, passes=n_iterations, alpha=a, eta=b, workers=w)
    # get coherence
    coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    return coherence

In [31]:
current_cost = get_cost(current_state)

KeyboardInterrupt: 

Traceback (most recent call last):
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\queues.py", line 241, in _feed
    send_bytes(obj)
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\connection.py", line 290, in _send_bytes
    nwritten, err = ov.GetOverlappedResult(True)
BrokenPipeError: [WinError 109] The pipe has been ended
Traceback (most recent call last):


NameError: name 'ldamodel' is not defined

  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\queues.py", line 241, in _feed
    send_bytes(obj)
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\connection.py", line 280, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=True)
BrokenPipeError: [WinError 232] The pipe is being closed
Traceback (most recent call last):
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\queues.py", line 241, in _feed
    send_bytes(obj)
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\mwhuhn\Anaconda3\envs\forum\lib\multiprocessing\connection.py", line 280, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=True)
BrokenPipeError: [WinError

In [ ]:
while current_temp > final_temp:
    neighbor = get_neighbor(current_state, 20, .25, lower_bounds, upper_bounds)

    # Check if neighbor is best so far
    neighbor_cost = get_cost(neighbor)
    cost_diff = current_cost - neighbor_cost

    # if the new solution is better, accept it
    if cost_diff > 0:
        solution = neighbor
        current_cost = neighbor_cost

    # if the new solution is not better, accept it with a
    # probability of e^(-cost/temp)
    else:
        if random.uniform(0, 1) < math.exp(cost_diff / current_temp):
            solution = neighbor
            current_cost = neighbor_cost

    # decrement the temperature
    current_temp -= alpha

In [ ]:
# explore solution